# Setup

## Instalar de bibliotecas

In [ ]:
! pip install implicit tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importar pacotes

In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
import implicit
import tabulate

# Recomendação baseada em conteúdo

## Criar dados

### Itens

In [ ]:
itens = {
    "Vingadores": [0, 1, 1, 1, 0], 
    "Titanic": [1, 0, 0, 0, 1],
    "Marley e Eu": [1, 0, 0, 0, 1],
    "Coringa": [1, 1, 0, 0, 0],
    "Homem-Aranha": [0, 1, 1, 1, 0],
    "O lado bom da vida": [1, 0, 0, 0, 1]
}
feature_names = ["Drama", "Super heróis", "Ação", "Comédia", "Romance"]

In [ ]:
item_df = pd.DataFrame(data=itens, index=feature_names).T
item_df

,Drama,Super heróis,Ação,Comédia,Romance
Vingadores,0,1,1,1,0
Titanic,1,0,0,0,1
Marley e Eu,1,0,0,0,1
Coringa,1,1,0,0,0
Homem-Aranha,0,1,1,1,0
O lado bom da vida,1,0,0,0,1


### Preferência de pessoa usuária

In [ ]:
user_preference = {
    "Vingadores": 1, 
    "Titanic": -1,
    "Marley e Eu": -1,
    "Coringa": 1,
    "Homem-Aranha": np.nan,
    "O lado bom da vida": np.nan
}

In [ ]:
user_df = pd.DataFrame(data=user_preference, index=["Pessoa Usuária"]).T
user_df

,Pessoa Usuária
Vingadores,1.0
Titanic,-1.0
Marley e Eu,-1.0
Coringa,1.0
Homem-Aranha,NaN
O lado bom da vida,NaN


## Perfil da pessoa usuária

In [ ]:
item_df.multiply(user_df["Pessoa Usuária"], axis=0)

,Drama,Super heróis,Ação,Comédia,Romance
Vingadores,0.0,1.0,1.0,1.0,0.0
Titanic,-1.0,-0.0,-0.0,-0.0,-1.0
Marley e Eu,-1.0,-0.0,-0.0,-0.0,-1.0
Coringa,1.0,1.0,0.0,0.0,0.0
Homem-Aranha,NaN,NaN,NaN,NaN,NaN
O lado bom da vida,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_profile = item_df.multiply(user_df["Pessoa Usuária"], axis=0).sum()
user_profile

Drama          -1.0
Super heróis    2.0
Ação            1.0
Comédia         1.0
Romance        -2.0
dtype: float64

## Recomendação

In [ ]:
item_df.multiply(user_profile)

,Drama,Super heróis,Ação,Comédia,Romance
Vingadores,-0.0,2.0,1.0,1.0,-0.0
Titanic,-1.0,0.0,0.0,0.0,-2.0
Marley e Eu,-1.0,0.0,0.0,0.0,-2.0
Coringa,-1.0,2.0,0.0,0.0,-0.0
Homem-Aranha,-0.0,2.0,1.0,1.0,-0.0
O lado bom da vida,-1.0,0.0,0.0,0.0,-2.0


In [ ]:
item_df.multiply(user_profile).sum(axis=1)

Vingadores            4.0
Titanic              -3.0
Marley e Eu          -3.0
Coringa               1.0
Homem-Aranha          4.0
O lado bom da vida   -3.0
dtype: float64

## Exercício

In [ ]:
users_preferences = {
    "Filme A": [1, np.nan, 1], 
    "Filme B": [np.nan, 1, np.nan],
    "Filme C": [1, -1, 1],
    "Filme D": [-1, np.nan, np.nan],
    "Filme E": [np.nan, 1, np.nan]
}
users = [f"Pessoa {i}" for i in range(1,4)]
users_df = pd.DataFrame(data=users_preferences, index=users).T
users_df

,Pessoa 1,Pessoa 2,Pessoa 3
Filme A,1.0,NaN,1.0
Filme B,NaN,1.0,NaN
Filme C,1.0,-1.0,1.0
Filme D,-1.0,NaN,NaN
Filme E,NaN,1.0,NaN


In [ ]:
itens = {
    "Filme A": [0, 1, 1, 1, 0], 
    "Filme B": [1, 0, 0, 0, 1],
    "Filme C": [0, 0, 1, 1, 0],
    "Filme D": [0, 0, 0, 1, 1],
    "Filme E": [1, 1, 1, 0, 0]
}
items_df = pd.DataFrame(data=itens, index=feature_names).T
items_df

,Drama,Super heróis,Ação,Comédia,Romance
Filme A,0,1,1,1,0
Filme B,1,0,0,0,1
Filme C,0,0,1,1,0
Filme D,0,0,0,1,1
Filme E,1,1,1,0,0


In [ ]:
users_df["Pessoa 1"]

Filme A    1.0
Filme B    NaN
Filme C    1.0
Filme D   -1.0
Filme E    NaN
Name: Pessoa 1, dtype: float64

In [ ]:
items_df.multiply(users_df["Pessoa 1"], axis=0)

,Drama,Super heróis,Ação,Comédia,Romance
Filme A,0.0,1.0,1.0,1.0,0.0
Filme B,NaN,NaN,NaN,NaN,NaN
Filme C,0.0,0.0,1.0,1.0,0.0
Filme D,-0.0,-0.0,-0.0,-1.0,-1.0
Filme E,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_profile = items_df.multiply(users_df[f"Pessoa 1"], axis=0).sum()
user_profile

Drama           0.0
Super heróis    1.0
Ação            2.0
Comédia         1.0
Romance        -1.0
dtype: float64

In [ ]:
for i in range(1, 4):
  user_profile = items_df.multiply(users_df[f"Pessoa {i}"], axis=0).sum()
  rec = items_df.multiply(user_profile).sum(axis=1)
  print(f"Pessoa {i}")
  print(tabulate.tabulate(rec.reset_index().values))
  print()

Pessoa 1
-------  --
Filme A   4
Filme B  -1
Filme C   3
Filme D   0
Filme E   3
-------  --

Pessoa 2
-------  --
Filme A   0
Filme B   3
Filme C  -1
Filme D   0
Filme E   3
-------  --

Pessoa 3
-------  -
Filme A  5
Filme B  0
Filme C  4
Filme D  2
Filme E  3
-------  -



# Filtragem colaborativa

## Gerando dados

In [ ]:
def generate_data(n_users=10_000):
  n_users = int(n_users/2)
  feedback = {
      "Vingadores": [np.random.choice([1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)] + [np.random.choice([-1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)], 
      "Titanic": [np.random.choice([-1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)] + [np.random.choice([1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)],
      "Marley e Eu": [np.random.choice([-1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)] + [np.random.choice([1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)],
      "Coringa": [np.random.choice([1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)] + [np.random.choice([-1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)],
      "Homem-Aranha": [np.random.choice([1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)] + [np.random.choice([-1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)],
      "O lado bom da vida": [np.random.choice([-1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)] + [np.random.choice([1, np.nan])*np.random.randint(1, 6) for _ in range(n_users)]
  }
  users = [f"Pessoa {i}" for i in range(1, 2*n_users+1)]
  R = pd.DataFrame(data=feedback, index=users)
  return R

In [ ]:
R = generate_data()
R

,Vingadores,Titanic,Marley e Eu,Coringa,Homem-Aranha,O lado bom da vida
Pessoa 1,NaN,NaN,NaN,NaN,5.0,NaN
Pessoa 2,1.0,NaN,NaN,1.0,NaN,-1.0
Pessoa 3,NaN,NaN,NaN,4.0,5.0,NaN
Pessoa 4,NaN,NaN,NaN,NaN,4.0,-3.0
Pessoa 5,3.0,-3.0,NaN,NaN,NaN,-2.0
...,...,...,...,...,...,...
Pessoa 9996,-1.0,5.0,NaN,NaN,NaN,NaN
Pessoa 9997,NaN,2.0,NaN,-5.0,-4.0,5.0
Pessoa 9998,-4.0,NaN,NaN,-3.0,NaN,NaN
Pessoa 9999,NaN,NaN,NaN,-3.0,NaN,3.0


In [ ]:
R.isnull().sum().sum()

30037

In [ ]:
R_sparse = (
    R
    .reset_index()
    .melt(id_vars=["index"])
    .rename(
        {
            "index": "user",
            "variable": "movie",
            "value": "feedback"
        },
        axis=1
    )
)

# Transform to category data type
R_sparse["user"] = R_sparse["user"].astype("category")
R_sparse["movie"] = R_sparse["movie"].astype("category")

# Get ids
R_sparse["userid"] = R_sparse["user"].cat.codes
R_sparse["movieid"] = R_sparse["movie"].cat.codes

# Drop nan
R_sparse.dropna(inplace=True)
R_sparse

,user,movie,feedback,userid,movieid
1,Pessoa 2,Vingadores,1.0,1112,5
4,Pessoa 5,Vingadores,3.0,4445,5
6,Pessoa 7,Vingadores,3.0,6667,5
7,Pessoa 8,Vingadores,5.0,7778,5
9,Pessoa 10,Vingadores,3.0,1,5
...,...,...,...,...,...
59987,Pessoa 9988,O lado bom da vida,1.0,9987,3
59990,Pessoa 9991,O lado bom da vida,3.0,9991,3
59992,Pessoa 9993,O lado bom da vida,2.0,9993,3
59996,Pessoa 9997,O lado bom da vida,5.0,9997,3


In [ ]:
R_sparse.sort_values("user")

,user,movie,feedback,userid,movieid
40000,Pessoa 1,Homem-Aranha,5.0,0,1
10009,Pessoa 10,Titanic,-3.0,1,4
9,Pessoa 10,Vingadores,3.0,1,5
50009,Pessoa 10,O lado bom da vida,-4.0,1,3
30099,Pessoa 100,Coringa,5.0,2,0
...,...,...,...,...,...
49996,Pessoa 9997,Homem-Aranha,-4.0,9997,1
9997,Pessoa 9998,Vingadores,-4.0,9998,5
39997,Pessoa 9998,Coringa,-3.0,9998,0
39998,Pessoa 9999,Coringa,-3.0,9999,0


In [ ]:
R_csr = sparse.csr.csr_matrix((R_sparse["feedback"].astype(float), (R_sparse["userid"], R_sparse["movieid"])))

In [ ]:
R_csr

<10000x6 sparse matrix of type '<class 'numpy.float64'>'
	with 29963 stored elements in Compressed Sparse Row format>

## Treino do modelo

In [ ]:
als = implicit.als.AlternatingLeastSquares(
    factors=20, # número de dimensões
    regularization=0.01, # como é um modelo de regressão linear, podemos usar regularização
    iterations=100
)

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:27: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


In [ ]:
als.fit(R_csr,)

  0%|          | 0/100 [00:00<?, ?it/s]

## Recomendação

In [ ]:
R_sparse.drop_duplicates(subset=["movie", "movieid"])[["movie", "movieid"]]

,movie,movieid
1,Vingadores,5
10004,Titanic,4
20005,Marley e Eu,2
30001,Coringa,0
40000,Homem-Aranha,1
50001,O lado bom da vida,3


In [ ]:
als.recommend_all(user_items=R_csr, N=6)

array([[4, 2, 5, 3, 0, 1],
       [1, 0, 2, 5, 4, 3],
       [4, 5, 3, 2, 1, 0],
       ...,
       [2, 5, 4, 3, 1, 0],
       [4, 2, 1, 3, 5, 0],
       [2, 4, 1, 5, 3, 0]], dtype=int32)

## Embeddings

In [ ]:
als.similar_items(5, N=6)

(array([5, 1, 0, 4, 2, 3], dtype=int32),
 array([ 1.0000001 ,  0.00591782,  0.0041391 , -0.00185744, -0.00260336,
        -0.00412723], dtype=float32))

In [ ]:
https://matheusjorge.github.io/